In [ ]:
import pandas as pd
import geopandas as gpd
import ast
### Generating a single geojson file with all the data for paul
sommarioni_people = pd.read_json('../../named_entity_standardisation/1808_Sommarioni_standardised/people_sommarioni_dataset.json').drop(columns=['edited', 'merged_ids'])
sommarioni_people['uid'] = sommarioni_people['uid'].astype(int)
sommarioni_people['nucleus_uid'] = sommarioni_people['nucleus_uid'].astype(int)
sommarioni_people['parcel_array'] = sommarioni_people.parcel_ids.apply(lambda v: ast.literal_eval('[' + v+']'))
people_duplicated = []
for _, r in sommarioni_people.iterrows():
    for parcel_id in r.parcel_array:
        new_r = r.copy()
        new_r['parcel_id'] = parcel_id
        people_duplicated.append(new_r)


In [ ]:
sommarioni_std_full = pd.read_json('sommarioni_text_data_20250318_internal.json')
sommarioni_std_full

In [ ]:
people_flattened = pd.DataFrame(people_duplicated).drop(columns=['parcel_array', 'parcel_ids'])
rename_owner_dict = {v:'own_'+v for v in people_flattened.columns.to_list()}
people_flattened = people_flattened.rename(columns=rename_owner_dict)
sommarioni_text_people = sommarioni_std_full.merge(people_flattened, left_on='unique_id', right_on='own_parcel_id', how='left').drop(columns=['own_parcel_id'])

In [ ]:
geometries_df = gpd.read_file('sommarioni_geometries_internal_20250318.geojson')

In [ ]:
sommarioni_text_people.geometry_id = sommarioni_text_people.geometry_id.astype(str)

In [ ]:
geometries_group = geometries_df.groupby('geometry_id')
geometries_group

In [ ]:
sommarioni_text_people_group = sommarioni_text_people.groupby('geometry_id')

In [ ]:
# building JSON with the corresponding group
text_drop_cols = ['geometry_id', 'unique_id']
sommarioni_text_people_group_dict = sommarioni_text_people_group.apply(lambda x: x.drop().to_json(orient='records')).to_dict()
sommarioni_text_people_group_dict

In [ ]:
final_file_name = f'../../1808_Sommarioni/aggregated/sommarioni_geometries_function_and_people_standardized_20250318.geojson'

to_write = geometries_df.merge(sommarioni_text_people, on='geometry_id', how='left').rename(columns={'parcel_number_x': 'parcel_number'}).drop(columns='parcel_number_y').to_json(indent=2)
with open(final_file_name, 'w', encoding='utf-8') as file:
     file.write(to_write)